# Using LlamaIndex to Automate the fine-tuning of GPT-3.5-turbo on source documents

Primarly Extended from [this](https://colab.research.google.com/drive/1vWeJBXdFEObuihO7Z8ui2CAYkdHQORqo?usp=sharing) notebook, we'll take a look at how we can wrap this process into Chainlit and have our own dynamic fine-tuning machine!

In [1]:
!pip install -q -U llama-index pypdf sentence-transformers ragas openai

In [2]:
import os
from getpass import getpass

openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
!curl https://jaydixit.com/files/PDFs/TheultimateHitchhikersGuide.pdf --output hitchhikers.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3322k  100 3322k    0     0   720k      0  0:00:04  0:00:04 --:--:--  721k


In [4]:
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator

documents = SimpleDirectoryReader(
    input_files=["hitchhikers.pdf"]
).load_data()

# Shuffle the documents
import random

random.seed(42)
random.shuffle(documents)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context from a "
    "report on climate change and the oceans, formulate "
    "a single question that captures an important fact from the "
    "context. Restrict the question to the context information provided."
)

dataset_generator = DatasetGenerator.from_documents(
    documents[:50],
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
)

### Generative Questions with `gpt-3.5-turbo`

We can use the `generate_questions_from_nodes()` method of our dataset generator to produce a number of questions that will be used to fine-tune!

> NOTE: This cell will take ~30s-2min.

In [5]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


Let's take a peek and see what was created!

In [11]:
questions[0]

'What did Zaphod find on the external monitor screens?'

Now we can save our questions into a text file for later use.

In [12]:
with open("train_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

### Evaluation Generator

Let's generate questions from a different segment of our documents in order to build a robust test for our RAQA.

In [13]:
dataset_generator = DatasetGenerator.from_documents(
    documents[
        50:
    ],  # since we generated ~1 question for 40 documents, we can skip the first 40
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
)

Again, we'll use `gpt-3.5-turbo` to generate some questions!

In [26]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


Now we can save our results for evaluations later!

In [28]:
with open("eval_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

### Evaluating base `gpt-3.5-turbo`

We'll load up our evaluation questions and get to it!

In [29]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

This next cell is constructing our `VectorIndex` so we can move onto testing the base model.

In [30]:
from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3), context_window=2048
)

index = VectorStoreIndex.from_documents(documents, service_context=gpt_35_context)

query_engine = index.as_query_engine(similarity_top_k=2)

Here is where we're actually putting the model to the test!

In [31]:
documents

[Document(id_='03d4712e-d2f1-4c8f-b498-7c5c007b3ec2', embedding=None, metadata={'page_label': '93', 'file_name': 'hitchhikers.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='17cd8bab8f34f12684848ae951925ad9f195e9525932e5821a2ea6a7c4edbe6e', text='Light \xa0on \xa0the \xa0bridge \xa0sank. \xa0Pinpoints \xa0of \xa0light \xa0played \xa0across \xa0the \xa0\nconsoles \xa0and \xa0reflected \xa0in \xa0four \xa0pairs \xa0of \xa0eyes \xa0that \xa0stared \xa0up \xa0at \xa0the \xa0\nexternal \xa0monitor \xa0screens. \xa0  \xa0\nThere \xa0was \xa0absolutely \xa0nothing \xa0on \xa0them. \xa0  \xa0\n"Recognize \xa0that?" \xa0whispered \xa0Zaphod. \xa0  \xa0\nFord \xa0frowned. \xa0  \xa0\n"Er, \xa0no," \xa0he \xa0said. \xa0  \xa0\n"What \xa0do \xa0you \xa0see?" \xa0  \xa0\n"Nothing." \xa0  \xa0\n"Recognize \xa0it?" \xa0  \xa0\n"What \xa0are \xa0you \xa0talking \xa0about?" \xa0  \xa0\n"We\'re \xa0in \xa0the \xa0Horsehead \xa0Nebula. \xa0One \xa0whole \xa0

In [33]:
contexts = []
answers = []

for question in questions:
    print("Question: ", question)

    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

Question:  What do Ford and Arthur observe happening to the masses below them?
Question:  What was Arthur's reaction to not knowing what something meant to him?
Question:  What degrees does Gail Andrews hold in mathematics and astrophysics?
Question:  What did Tricia McMillan do after staring at the security video camera?
Question:  What is the Answer to the Ultimate Question of Life, the Universe, and Everything according to the provided context?
Question:  What is the statement about the possibility of knowing two things about the same universe?
Question:  What was the disastrous failure of the Starship Titanic shortly after its launch?
Question:  What is the name of the spaceship that regularly commands the most actual astonishment?
Question:  Question: What did Zaphod find in his pocket that he put on before climbing down the building?
Question:  What is Zaphod's response when asked if he is going to be put into the Vortex?
Question:  Question: What does the Restaurant resemble whe

Now that we've tested our model - let's evaluate it to see how it performed!

We're testing our model with the `ragas` framework - found [here](https://github.com/explodinggradients/ragas)

You'll notice that we're testing two primary metrics:

- [`answer_relevancy`](https://github.com/explodinggradients/ragas/blob/a55c3be8b2389501c5c761df9070126027a4d1d6/src/ragas/metrics/answer_relevance.py#L32): This measures how relevant is the generated answer to the prompt. If the generated answer is incomplete or contains redundant information the score will be low. This is quantified by working out the chance of an LLM generating the given question using the generated answer. Values range (0,1), higher the better.
- [`faithfulness`](https://github.com/explodinggradients/ragas/blob/a55c3be8b2389501c5c761df9070126027a4d1d6/src/ragas/metrics/faithfulnes.py#L63): This measures the factual consistency of the generated answer against the given context. This is done using a multi step paradigm that includes creation of statements from the generated answer followed by verifying each of these statements against the context. The answer is scaled to (0,1) range. Higher the better.

Read more about their implementations [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)

Again, these cells might take some time to complete - be patient!

In [34]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

/Users/jay/miniconda3/envs/llmops-course/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


evaluating with [answer_relevancy]


100%|██████████| 3/3 [01:16<00:00, 25.43s/it]


evaluating with [faithfulness]


100%|██████████| 3/3 [04:53<00:00, 97.87s/it] 


{'ragas_score': 0.8425, 'answer_relevancy': 0.9211, 'faithfulness': 0.7762}


In [35]:
base_eval = {'ragas_score': 0.8230, 'answer_relevancy': 0.9308, 'faithfulness': 0.7375}

ds

Dataset({
    features: ['question', 'answer', 'contexts'],
    num_rows: 40
})

### Leveraging `gpt-4` to improve our `gpt-3.5-turbo` base model!

In [17]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [24]:
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [25]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=gpt_4_context)

query_engine = index.as_query_engine(similarity_top_k=2)

Again, this process will take a few minutes. 

While this is a powerful technique - it is unfortunately quite slow.

In [26]:
for question in questions:
    response = query_engine.query(question)

### Creating the fine-tuning dataset

Now that we have a number of fine-tuning events from our `OpenAIFineTuningHandler()`, let's save them to a `.jsonl` file - the expected format for fine-tuning `gpt-3.5-turbo`!

In [27]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

Wrote 46 examples to finetuning_events.jsonl


In [29]:
import openai
file_response = openai.File.create(file=open("finetuning_events.jsonl", "rb"), purpose='fine-tune')

In [30]:
file_response

<File file id=file-6adVuM8Pm7UdksqYKxVklSJg at 0x7f2dd6f1cdd0> JSON: {
  "object": "file",
  "id": "file-6adVuM8Pm7UdksqYKxVklSJg",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 200354,
  "created_at": 1692905441,
  "status": "uploaded",
  "status_details": null
}

In [31]:
import time

response = None

while not response:
  try:
    response = openai.FineTuningJob.create(training_file=file_response.id, model="gpt-3.5-turbo")
  except:
    time.sleep(5)

In [32]:
response

<FineTuningJob fine_tuning.job id=ftjob-SUD1CJJwBbTvrkV2G5K6bkUY at 0x7f2dd6f1d250> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-SUD1CJJwBbTvrkV2G5K6bkUY",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692905472,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-VrtlDUDw6aAbBGBDNn1W797P",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-6adVuM8Pm7UdksqYKxVklSJg",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [33]:
training_id = response.id

In [34]:
openai.FineTuningJob.retrieve(training_id)

<FineTuningJob fine_tuning.job id=ftjob-SUD1CJJwBbTvrkV2G5K6bkUY at 0x7f2dd710a750> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-SUD1CJJwBbTvrkV2G5K6bkUY",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692905472,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-VrtlDUDw6aAbBGBDNn1W797P",
  "result_files": [],
  "status": "running",
  "validation_file": null,
  "training_file": "file-6adVuM8Pm7UdksqYKxVklSJg",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [35]:
from IPython.display import clear_output

while openai.FineTuningJob.retrieve(training_id).status == "running":
  clear_output(wait=True)
  time.sleep(5)
  print(openai.FineTuningJob.list_events(id=training_id, limit=10))

print("Done!")

{
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-ioAEPvMGT9I4dJz2hVw9EXc0",
      "created_at": 1692906144,
      "level": "info",
      "message": "Fine-tuning job successfully completed",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-UtuXIbtM5Ht7tZTnYFOunmDF",
      "created_at": 1692906142,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:ox::7rAVhh9B",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-CyQJQN8bxXIOYOVWCKztOqeG",
      "created_at": 1692906130,
      "level": "info",
      "message": "Step 130/138: training loss=0.00",
      "data": {
        "step": 130,
        "train_loss": 0.0009155995794571936,
        "train_mean_token_accuracy": 1.0
      },
      "type": "metrics"
    },
    {
      "object": "fine_tuning.job.event",
      "

In [36]:
openai.FineTuningJob.retrieve(training_id)

<FineTuningJob fine_tuning.job id=ftjob-SUD1CJJwBbTvrkV2G5K6bkUY at 0x7f2dd72075f0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-SUD1CJJwBbTvrkV2G5K6bkUY",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692905472,
  "finished_at": 1692906144,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:ox::7rAVhh9B",
  "organization_id": "org-VrtlDUDw6aAbBGBDNn1W797P",
  "result_files": [
    "file-lb9W8pRtbjI5JP4YoZ2AeKtk"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-6adVuM8Pm7UdksqYKxVklSJg",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 159942
}

In [37]:
ft_model_id = openai.FineTuningJob.retrieve(training_id).fine_tuned_model

### Evaluating the fine-tuned model

Now that we've fine-tuned our model on the `gpt-4` enhanced question answers - let's see how it performs on our `raga` evaluation!

In [38]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager


ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model=ft_model_id, temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [39]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [40]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [41]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [42]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

evaluating with [answer_relevancy]


100%|██████████| 3/3 [01:03<00:00, 21.28s/it]


evaluating with [faithfulness]


100%|██████████| 3/3 [04:22<00:00, 87.38s/it]


{'ragas_score': 0.8599, 'answer_relevancy': 0.9398, 'faithfulness': 0.7925}


In [53]:
ft_eval = {'ragas_score': 0.8599, 'answer_relevancy': 0.9398, 'faithfulness': 0.7925}

### Exploring Differences

Now we can compare the outputs of the two models!

In [43]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [44]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [45]:
print(questions[12])

What did Arthur see when he moved back and looked again at the solid wall of blue ice?


In [46]:
from llama_index.response.notebook_utils import display_response
from llama_index import ServiceContext
from llama_index.llms import OpenAI


gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [47]:
query_engine = index.as_query_engine(service_context=gpt_35_context)

response = query_engine.query(questions[12])

display_response(response)

**`Final Response:`** Arthur saw the sky when he moved back and looked again at the solid wall of blue ice.

In [50]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI


ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model=ft_model_id, temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [51]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[12])

display_response(response)

**`Final Response:`** Arthur saw something within the frozen depths.

In [60]:
metric_list = ["answer_relevancy", "faithfulness"]

for metric in metric_list:
  print("Base model", metric, ":", base_eval[metric])
  print("Fine-tuned model", metric, ":", ft_eval[metric])
  print(f"Improvement {metric} : {(ft_eval[metric] - base_eval[metric])*100:.2f}%")
  print()

Base model answer_relevancy : 0.9308
Fine-tuned model answer_relevancy : 0.9398
Improvement answer_relevancy : 0.90%

Base model faithfulness : 0.7375
Fine-tuned model faithfulness : 0.7925
Improvement faithfulness : 5.50%

